# INELASTIC TOY NETWORK STA

This file first shows how to run an inelastic Static Traffic Assignment in dyntapy. It also shows examples of how to run an inelastic assignment with tolls.

1. At first, you import all the necessary packages and files from mostly dyntapy. 

In [1]:
import warnings
warnings.filterwarnings('ignore') # hide warnings
import numpy as np
import os
import sys
sys.path.append("../../..")

from pickle import dump
from dyntapy.supply_data import get_toy_network, relabel_graph
from dyntapy.demand_data import add_centroids, od_graph_from_matrix
from dyntapy.assignments import StaticAssignment
from dyntapy.visualization import show_network, show_demand
from dyntapy.toll import create_toll_object

2. Some toy networks are already made available in the original toolbox, so in the example, Cascetta is used. The first steps are always the same: 

- Create or retrieve a network
- Create and add centroids to network → adds connectors automatically
- Create or load an OD matrix (with the demand) onto the centroids

  These steps can also be visualised in the notebooks by fully running the next block of code. There are now links visible (roads and connectors), green centroids and for both elements when hovering with the cursor over the segments there are also some characteristics displayed in a box like the link ID, the capacity, the free-flow speed or demand between each location in the demand plot.

In [2]:
# 1. Retrieve network 
g = get_toy_network('cascetta')

# 2. Create and add centroids
centroid_x = np.array([2+np.sqrt(2), 5, np.sqrt(2)])
centroid_y = np.array([-0.5, np.sqrt(2), np.sqrt(2)])
g = add_centroids(g, centroid_x, centroid_y, euclidean=True)
# also adds connectors automatically
g = relabel_graph(g)  # adding link and node ids, connectors and centroids
show_network(g, euclidean=True, notebook=True)

# 3. Create OD and load onto centroids 
od_matrix = np.zeros(9).reshape((3, 3))
od_matrix[0, 1] = 100
od_matrix[2, 1] = 100
od_graph = od_graph_from_matrix(od_matrix, centroid_x, centroid_y)
show_demand(od_graph, euclidean=True, notebook=True)

3. Now all the elements are there to run an STA in the next block of code. When this assignment is performed, there is now a loaded network visible with the traffic flows as an additional characteristic. 

In [3]:
assignment = StaticAssignment(g, od_graph)
result = assignment.run('dial_b')
print('dial_b ran successfully')
show_network(g, flows = result.flows, notebook=True, show_nodes=False, euclidean=True)


# EXTRA: all static assignments return a result object that follows the same structure
print(result.__dict__.keys())

init passed successfully
initial loading starts 
dial_b ran successfully


dict_keys(['link_costs', 'flows', 'origins', 'destinations', 'origin_flows', 'destination_flows', 'skim', 'gap_definition', 'gap', 'od_flows'])


    Congratulations, you have now run your first STA using dyntapy!

4. You can now add toll and explore different tolling schemes. Here you will see examples of all tolling methods, including single link, cordon- and zone-based tolling schemes. To get to know more about what these mean, consult the literature or the additional paper. What is shown in the code are the three main elements per scheme: the links where the toll is put on, the value of this toll and the toll_object, which is the additional element that should be given to the STA. Decide on the links and the toll values and afterwards, please run the StaticAssignment again in the next block of code. You now have the distribution of flows with the toll cost on top of the normal BPR link costs.


In [4]:
# Let's first consider a single toll, on link 9. This link starts at the bottom and goes to the right. 
toll_method = 'single'
toll_link_id = 9 
toll_value = 10

# Create toll object
toll_object = create_toll_object(g, toll_method, toll_link_id, toll_value)

# Run STA with toll object, verify that the result does not use the tolled link because of its high costs. 
assignment2 = StaticAssignment(g,od_graph, toll_object)
result2 = assignment2.run('dial_b')
show_network(g, flows = result2.flows, notebook=True, show_nodes=False, euclidean=True)


init passed successfully
initial loading starts 


In [5]:
# Other supported tolling schemes are cordon or zone. 
# For the toy network there is not really a diffence, but let's use both. 
# We consider links 9 (bottom to right) and link 10 (bottom to top) to form the cordon. 

toll_method = 'cordon'
toll_link_id = [9, 10]
toll_value = 10
toll_object2 = create_toll_object(g, toll_method, toll_link_id, toll_value)
assignment3 = StaticAssignment(g,od_graph, toll_object2)
result3 = assignment3.run('dial_b')
show_network(g, flows = result3.flows, notebook=True, show_nodes=False, euclidean=True)
# verify that the result does not use the cordon links because of their high costs. 

init passed successfully
initial loading starts 


In [6]:
# Let's now use the same links, but consider them as a zone. The toll value on each link is defined as the 
# length of the link times the provided toll value per km. 
toll_method = 'zone'
toll_link_id = [9, 10]
toll_value_per_km = 0.5
toll_object3 = create_toll_object(g, toll_method, toll_link_id, toll_value_per_km)
assignment4 = StaticAssignment(g,od_graph, toll_object3)
result4 = assignment4.run('dial_b')
show_network(g, flows = result4.flows, notebook=True, show_nodes=False, euclidean=True)

init passed successfully
initial loading starts 


In [7]:
# In the case of cordon tolling, there is also support to set different toll values on different links. 
# You should provide an equally long list of toll values compared to the list with link ids. 
toll_method = 'cordon'
toll_link_id = [5, 9, 10]
toll_value = [10, 20, 30]
toll_object4 = create_toll_object(g, toll_method, toll_link_id, toll_value)
assignment5 = StaticAssignment(g,od_graph, toll_object4)
result5 = assignment5.run('dial_b')
show_network(g, flows = result5.flows, notebook=True, show_nodes=False, euclidean=True)

init passed successfully
initial loading starts 


    Congratulations, you have now implemented toll on a toy network! 

-----------------------------------------------------------------------------------------------------------------------


5. The last step is preparing for the Heeds iterations. The outer loop will decide on the best toll value, by ‘randomly’ trying toll values within a given range. To do this, the full network (containing roads, centroids and connectors) with the OD matrix should preferably be saved to avoid reloading these files each iteration. Because both remain static in an inelastic use-case, they are saved with a given path. The method to do so is indicated in the last block of the file.

In [8]:
HERE = os.path.dirname(os.path.realpath("__file__"))
HEEDS_data_path = HERE+ os.path.sep+os.pardir+os.path.sep+os.pardir+os.path.sep+'data_map' + os.path.sep + "HEEDS_input"

network_path = HEEDS_data_path + os.path.sep + 'network_with_centroids' + os.path.sep + "inelastic_toy"
graph_path = HEEDS_data_path + os.path.sep + "od_graph" + os.path.sep + "inelastic_toy"

with open(network_path, 'wb') as network_file:
    dump(g, network_file)
    print(f'network saved at f{network_path}')
with open(graph_path, 'wb') as f:
    dump(od_graph,f)
    print(f'od_graph saved at f{graph_path}')

network saved at fC:\Users\silky\Documents\School\KU_Leuven\Master\YEAR_2\IP2\IP2\toll_optimization\case_studies\toy_network_inelastic_demand\..\..\data_map\HEEDS_input\network_with_centroids\inelastic_toy
od_graph saved at fC:\Users\silky\Documents\School\KU_Leuven\Master\YEAR_2\IP2\IP2\toll_optimization\case_studies\toy_network_inelastic_demand\..\..\data_map\HEEDS_input\od_graph\inelastic_toy


    Congratulations, you are now ready to go to the next step: HEEDS!